Activity 2: K-nn classification with credit data

 I will be using knn on job,credit.amount,duration to classify variables based on credit.risks. I will do so with a range of K and a range of sample size.
 

I have also analyzed the results using default data, data with intutive normalization and data with min-max normalization.

Steps

Loading libraries

In [4]:
library(ggplot2)
library(dplyr)
library(FNN)

Warning message:
"package 'ggplot2' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'FNN' was built under R version 3.3.3"

Loading the data

In [5]:
set.seed(2000)

In [6]:
credit.data=read.csv(file="german_credit_data1.csv", header=TRUE, sep=",")
head(credit.data) 
summary(credit.data)

X.,Age,Sex,Job,Housing,Saving.accounts,Checking.account,Credit.amount,Duration,Purpose,Credit.Risks
0,67,male,2,own,NA,little,1169,6,radio/TV,1
1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,49,male,1,own,little,NA,2096,12,education,1
3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,53,male,2,free,little,little,4870,24,car,2
5,35,male,1,free,NA,NA,9055,36,education,1


       X.             Age            Sex           Job        Housing   
 Min.   :  0.0   Min.   :19.00   female:310   Min.   :0.000   free:108  
 1st Qu.:249.8   1st Qu.:27.00   male  :690   1st Qu.:2.000   own :713  
 Median :499.5   Median :33.00                Median :2.000   rent:179  
 Mean   :499.5   Mean   :35.55                Mean   :1.904             
 3rd Qu.:749.2   3rd Qu.:42.00                3rd Qu.:2.000             
 Max.   :999.0   Max.   :75.00                Max.   :3.000             
                                                                        
   Saving.accounts Checking.account Credit.amount      Duration   
 little    :603    little  :274     Min.   :  250   Min.   : 4.0  
 moderate  :103    moderate:269     1st Qu.: 1366   1st Qu.:12.0  
 quite rich: 63    rich    : 63     Median : 2320   Median :18.0  
 rich      : 48    NA's    :394     Mean   : 3271   Mean   :20.9  
 NA's      :183                     3rd Qu.: 3972   3rd Qu.:24.0  
              

Changing the column names to all lower case

Changing all digits to numeric

In [7]:
names(credit.data) <- tolower(names(credit.data))
credit.data$age <- as.numeric(gsub("[^[:digit:]]","",credit.data$age))
credit.data$credit.amount <- as.numeric(gsub("[^[:digit:]]","",credit.data$credit.amount))
credit.data$duration <- as.numeric(gsub("[^[:digit:]]","",credit.data$duration))
credit.data$credit.risks <- as.numeric(gsub("[^[:digit:]]","",credit.data$credit.risks))

Cleaning empty data

In [8]:
credit.data=na.omit(credit.data)
head(credit.data)
# credit.data[!complete.cases(credit.data),]

,x.,age,sex,job,housing,saving.accounts,checking.account,credit.amount,duration,purpose,credit.risks
2,1,22,female,2,own,little,moderate,5951,48,radio/TV,2
4,3,45,male,2,free,little,little,7882,42,furniture/equipment,1
5,4,53,male,2,free,little,little,4870,24,car,2
8,7,35,male,3,rent,little,moderate,6948,36,car,1
10,9,28,male,3,own,little,moderate,5234,30,car,2
11,10,25,female,2,rent,little,moderate,1295,12,car,2


# PART 1.  No normalization

In [9]:
summary(credit.data)

       x.             age            sex           job        housing   
 Min.   :  1.0   Min.   :19.00   female:168   Min.   :0.000   free: 65  
 1st Qu.:233.5   1st Qu.:26.00   male  :354   1st Qu.:2.000   own :349  
 Median :502.5   Median :31.50                Median :2.000   rent:108  
 Mean   :493.5   Mean   :34.89                Mean   :1.875             
 3rd Qu.:736.8   3rd Qu.:41.00                3rd Qu.:2.000             
 Max.   :999.0   Max.   :75.00                Max.   :3.000             
                                                                        
   saving.accounts checking.account credit.amount      duration    
 little    :412    little  :245     Min.   :  276   Min.   : 6.00  
 moderate  : 64    moderate:224     1st Qu.: 1298   1st Qu.:12.00  
 quite rich: 23    rich    : 53     Median : 2326   Median :18.00  
 rich      : 23                     Mean   : 3279   Mean   :21.34  
                                    3rd Qu.: 3971   3rd Qu.:26.75  
        

In [16]:
n.points <- nrow(credit.data)
sampling.rate <- 0.8
credit.data.test.count<-n.points * (1 - sampling.rate)

In [17]:
train.rows=sample(1:n.points, sampling.rate * n.points,replace=FALSE)
# train.data <- subset(credit.data[train.rows, ], select = c(job,credit.amount,duration,credit.risks))
train.data <- subset(credit.data[train.rows, ], select = c(age,job,credit.amount,duration))
train.labels <- credit.data$credit.risks[train.rows]

In [18]:
test.rows=setdiff(1:n.points, train.rows) 
# test.data <- subset(credit.data[test.rows, ], select = c(job,credit.amount,duration,credit.risks))
test.data <- subset(credit.data[test.rows, ], select = c(age,job,credit.amount,duration))
test.labels <- credit.data$credit.risks[test.rows]

In [19]:
predicted.labels=knn(train.data, test.data,train.labels, k=3) 
 num.incorrect.labels <- sum(predicted.labels != test.labels) 
(num.incorrect.labels/length(test.rows))*100

[1] 38.09524

## We are getting very high misclassification rate of 38% for k=3 and sample size =0.8 on running knn on Un normalized data.

## This result is as expected since, some of the features are dominaing knn analysis

## Below we create a function to perform knn for sample size ranging from 0.5 to 0.9 with a step size of 0.1 and k taking value from 1 to 20 for each sample size

In [20]:
knnFun<-function(sampling.rate,n.points,observation,minified.obs){
    credit.data.test.count<-n.points * (1 - sampling.rate)
    train.rows=sample(1:n.points, sampling.rate * n.points,replace=FALSE)
    train.data <- subset(credit.data[train.rows, ], select = c(age,job,credit.amount,duration))
    train.labels <- credit.data$credit.risks[train.rows]
    test.rows=setdiff(1:n.points, train.rows) 
    test.data <- subset(credit.data[test.rows, ], select = c(age,job,credit.amount,duration))
    test.labels <- credit.data$credit.risks[test.rows]
    
    min=100.00000
    selected.k=1
    for (k in 1:20) {
        predicted.labels=knn(train.data, test.data,train.labels, k) 
        num.incorrect.labels <- sum(predicted.labels != test.labels) 
        missclafication.rate=(num.incorrect.labels/length(test.rows))*100
#         print(c(k,missclafication.rate))
            if(as.numeric(min) > as.numeric(missclafication.rate)) {
              min <- missclafication.rate
              selected.k=k
            }
        result=data.frame(k=k,mink=selected.k,misclassification.rate=missclafication.rate,train.size=sampling.rate)
        observation <- rbind(observation, result)
        }
#     sprintf("K= %f gives minimum misclaffication rate of %f fro sampling ratio of %f and %f", k, missclafication.rate,sampling.rate*100,(1-sampling.rate)*100)
        result2=data.frame(mink=selected.k,min.misclassification.rate=min,train.size=sampling.rate)
        minified.obs <- rbind(minified.obs, result2)
    return(list(observation,minified.obs))
}


### Calling the above finction on un normalized data

In [21]:
n.points <- nrow(credit.data)
# sampling.rate <- 0.7
minified.obs=data.frame(mink=double(),min.misclassification.rate=double(),train.size=double())
observation=data.frame(k=double(),mink=double(),misclassification.rate=double(),train.size=double(),minified.obs)
for(i in seq(0.5, 0.9, by = 0.1)){
#     sprintf("%f rate",i)
     
    x=knnFun(sampling.rate=i,n.points=nrow(credit.data),observation=observation,minified.obs)
    observation=data.frame(x[1])
    minified.obs=data.frame(x[2])
}


Recording all the boservation

In [22]:
observation

k,mink,misclassification.rate,train.size
1,1,46.74330,0.5
2,2,40.99617,0.5
3,2,43.29502,0.5
4,4,40.61303,0.5
5,4,42.14559,0.5
6,4,40.61303,0.5
7,4,40.99617,0.5
8,8,39.08046,0.5
9,8,40.22989,0.5
10,8,39.84674,0.5


### Tabulating analysis

The below table gives the minimum value of K in the range of 1 to 20 for which we got the minimum misclassification rate for a particular sample size

In [23]:
minified.obs
minified1=minified.obs

mink,min.misclassification.rate,train.size
8,39.08046,0.5
12,36.36364,0.6
11,35.03185,0.7
5,34.28571,0.8
20,35.84906,0.9


### Observation 1 : 

### Without normalization the best we could do is misclassification rate of 34.28% for k = 5 and sample size of 0.8

# Normlization 1 : 

# # Bringing credit amount duration  and credit risk to be in range of hundreds, so that no particular feature dominate the Distance.

In [49]:
credit.data.n1=credit.data
credit.data.n1$credit.amount=credit.data.n1$credit.amount/200
credit.data.n1$job=credit.data.n1$job*20
# credit.data.n1$credit.risks=credit.data.n1$credit.risks*10
head(credit.data.n1)

,x.,age,sex,job,housing,saving.accounts,checking.account,credit.amount,duration,purpose,credit.risks
2,1,22,female,40,own,little,moderate,29.755,48,radio/TV,2
4,3,45,male,40,free,little,little,39.410,42,furniture/equipment,1
5,4,53,male,40,free,little,little,24.350,24,car,2
8,7,35,male,60,rent,little,moderate,34.740,36,car,1
10,9,28,male,60,own,little,moderate,26.170,30,car,2
11,10,25,female,40,rent,little,moderate,6.475,12,car,2


Defining sampling rate and no. of total valid observations

In [50]:
summary(credit.data.n1)

       x.             age            sex           job        housing   
 Min.   :  1.0   Min.   :19.00   female:168   Min.   : 0.00   free: 65  
 1st Qu.:233.5   1st Qu.:26.00   male  :354   1st Qu.:40.00   own :349  
 Median :502.5   Median :31.50                Median :40.00   rent:108  
 Mean   :493.5   Mean   :34.89                Mean   :37.51             
 3rd Qu.:736.8   3rd Qu.:41.00                3rd Qu.:40.00             
 Max.   :999.0   Max.   :75.00                Max.   :60.00             
                                                                        
   saving.accounts checking.account credit.amount       duration    
 little    :412    little  :245     Min.   : 1.380   Min.   : 6.00  
 moderate  : 64    moderate:224     1st Qu.: 6.487   1st Qu.:12.00  
 quite rich: 23    rich    : 53     Median :11.633   Median :18.00  
 rich      : 23                     Mean   :16.394   Mean   :21.34  
                                    3rd Qu.:19.856   3rd Qu.:26.75  
  

Post normalization , we are getting mean of required data in the same range

In [51]:
n.points <- nrow(credit.data.n1)
sampling.rate <- 0.8
credit.data.test.count<-n.points * (1 - sampling.rate)

Sampling train data and labels

In [52]:
train.rows=sample(1:n.points, sampling.rate * n.points,replace=FALSE)
train.data <- subset(credit.data.n1[train.rows, ], select = c(age,job,credit.amount,duration))
train.labels <- credit.data.n1$credit.risks[train.rows]

Sampling test data and sample

In [53]:
test.rows=setdiff(1:n.points, train.rows) 
test.data <- subset(credit.data.n1[test.rows, ], select = c(age,job,credit.amount,duration))
test.labels <- credit.data.n1$credit.risks[test.rows]

Applying knn and obtaining predicted label

In [54]:
predicted.labels=knn(train.data, test.data,train.labels, k=3) 

checking the validity of results

In [55]:
 num.incorrect.labels <- sum(predicted.labels != test.labels) 

Calculating the misclassification rate

In [56]:
(num.incorrect.labels/length(test.rows))*100

[1] 36.19048

The intutive normalization didnot seem to produce much deviation<br>
Initially we got misclassification rate of 38%. <br>
Having an intutive normalization managed to reduce the misclassification rate by 2% to 36%

A function to peform the same analysis as above,but alltogether for varying k and sampling rate

### We create a function to perform knn for sample size ranging from 0.5 to 0.9 with a step size of 0.1 and k taking value from 1 to 20 for each sample size on data which is normalized using Min Max normalization

In [57]:
knnFun<-function(sampling.rate,n.points,observation,minified.obs){
    credit.data.test.count<-n.points * (1 - sampling.rate)
    train.rows=sample(1:n.points, sampling.rate * n.points,replace=FALSE)
    train.data <- subset(credit.data.n1[train.rows, ], select = c(age,job,credit.amount,duration))
    train.labels <- credit.data.n1$credit.risks[train.rows]
    test.rows=setdiff(1:n.points, train.rows) 
    test.data <- subset(credit.data.n1[test.rows, ], select = c(age,job,credit.amount,duration))
    test.labels <- credit.data.n1$credit.risks[test.rows]
    
    min=100.00000
    selected.k=1
    for (k in 1:20) {
        predicted.labels=knn(train.data, test.data,train.labels, k) 
        num.incorrect.labels <- sum(predicted.labels != test.labels) 
        missclafication.rate=(num.incorrect.labels/length(test.rows))*100
#         print(c(k,missclafication.rate))
            if(as.numeric(min) > as.numeric(missclafication.rate)) {
              min <- missclafication.rate
              selected.k=k
            }
        result=data.frame(k=k,mink=selected.k,misclassification.rate=missclafication.rate,train.size=sampling.rate)
        observation <- rbind(observation, result)
        }
#     sprintf("K= %f gives minimum misclaffication rate of %f fro sampling ratio of %f and %f", k, missclafication.rate,sampling.rate*100,(1-sampling.rate)*100)
        result2=data.frame(mink=selected.k,min.misclassification.rate=min,train.size=sampling.rate)
        minified.obs <- rbind(minified.obs, result2)
    return(list(observation,minified.obs))
}


Feeding different sample rates and k and tabulating the value in observations dataframe

In [58]:
n.points <- nrow(credit.data)
# sampling.rate <- 0.7
minified.obs=data.frame(mink=double(),min.misclassification.rate=double(),train.size=double())
observation=data.frame(k=double(),mink=double(),misclassification.rate=double(),train.size=double(),minified.obs)
for(i in seq(0.5, 0.9, by = 0.1)){
#     sprintf("%f rate",i)
     
    x=knnFun(sampling.rate=i,n.points=nrow(credit.data),observation=observation,minified.obs)
    observation=data.frame(x[1])
    minified.obs=data.frame(x[2])
}


#Tabulating all observations 

In [59]:
observation


k,mink,misclassification.rate,train.size
1,1,43.67816,0.5
2,2,42.14559,0.5
3,2,45.59387,0.5
4,2,42.14559,0.5
5,2,47.89272,0.5
6,2,43.67816,0.5
7,2,43.29502,0.5
8,8,40.22989,0.5
9,8,42.52874,0.5
10,10,38.31418,0.5


# Tabulating selected observations

In [60]:
minified.obs
minified2=minified.obs

mink,min.misclassification.rate,train.size
11,37.93103,0.5
4,36.84211,0.6
18,31.84713,0.7
18,37.14286,0.8
20,26.41509,0.9


# Observation 2 :

## After applying normalization ,the misclassification rate fell to 26.4% for train size of 90% nd k=20.<br>
Thus normalizaion is recommended. Without intutive normalization, best we could do was 34.28%

# Normalization 2

## II Trying out with min max normalization

In [61]:
normalize <- function(x) {
    return ((x - min(x)) / (max(x) - min(x)))
  }

In [62]:
credit.data.n2=credit.data
head(credit.data.n2)

,x.,age,sex,job,housing,saving.accounts,checking.account,credit.amount,duration,purpose,credit.risks
2,1,22,female,2,own,little,moderate,5951,48,radio/TV,2
4,3,45,male,2,free,little,little,7882,42,furniture/equipment,1
5,4,53,male,2,free,little,little,4870,24,car,2
8,7,35,male,3,rent,little,moderate,6948,36,car,1
10,9,28,male,3,own,little,moderate,5234,30,car,2
11,10,25,female,2,rent,little,moderate,1295,12,car,2


In [63]:
credit.data.n2.fl <- subset(credit.data.n2[, ], select = c(age,job,credit.amount,duration,credit.risks))
credit.data.n2.fl=as.data.frame(lapply(credit.data.n2.fl[,1:4], normalize))#Normalize alll feature except label
credit.data.n2.fl$credit.risks=credit.data.n2$credit.risks
head(credit.data.n2.fl)

age,job,credit.amount,duration,credit.risks
0.05357143,0.6666667,0.31270663,0.63636364,2
0.46428571,0.6666667,0.41910954,0.54545455,1
0.60714286,0.6666667,0.25314084,0.27272727,2
0.28571429,1.0000000,0.36764382,0.45454545,1
0.16071429,1.0000000,0.27319815,0.36363636,2
0.10714286,0.6666667,0.05614944,0.09090909,2


### Running knn for k=3 sample size = 0.8  after normalizing the data

In [70]:
train.data <- subset(credit.data.n2.fl[train.rows, ], select = c(age,job,credit.amount,duration))
train.labels <- credit.data.n2.fl$credit.risks[train.rows]
test.rows=setdiff(1:n.points, train.rows) 
test.data <- subset(credit.data.n2.fl[test.rows, ], select = c(age,job,credit.amount,duration))
test.labels <- credit.data.n2.fl$credit.risks[test.rows]
predicted.labels=knn(train.data, test.data,train.labels, k=3) 
 num.incorrect.labels <- sum(predicted.labels != test.labels) 
(num.incorrect.labels/length(test.rows))*100

[1] 40.95238

#### We get a misclassification rate of 40% after min max normalization. Which dosent prove to be of much benifit

### We create a function to perform knn for sample size ranging from 0.5 to 0.9 with a step size of 0.1 and k taking value from 1 to 20 for each sample size on data which is normalized using Min Max normalization

In [71]:

knnFun<-function(sampling.rate,n.points,observation,minified.obs){
    credit.data.test.count<-n.points * (1 - sampling.rate)
    train.rows=sample(1:n.points, sampling.rate * n.points,replace=FALSE)
    train.data <- subset(credit.data.n2.fl[train.rows, ], select = c(age,job,credit.amount,duration))
#     train.data.normalized=as.data.frame(lapply(train.data[,], normalize))#***********************
    train.labels <- credit.data.n2.fl$credit.risks[train.rows]
    test.rows=setdiff(1:n.points, train.rows) 
    test.data <- subset(credit.data.n2.fl[test.rows, ], select = c(age,job,credit.amount,duration))
#     test.data.normalized=as.data.frame(lapply(test.data[,], normalize))#*********************
    test.labels <- credit.data.n2.fl$credit.risks[test.rows]
    
    min=100.00000
    selected.k=1
    for (k in 1:20) {
        predicted.labels=knn(train.data, test.data,train.labels, k) 
        num.incorrect.labels <- sum(predicted.labels != test.labels) 
        missclafication.rate=(num.incorrect.labels/length(test.rows))*100
#         print(c(k,missclafication.rate))
            if(as.numeric(min) > as.numeric(missclafication.rate)) {
              min <- missclafication.rate
              selected.k=k
            }
        result=data.frame(k=k,mink=selected.k,misclassification.rate=missclafication.rate,train.size=sampling.rate)
        observation <- rbind(observation, result)
        }
#     sprintf("K= %f gives minimum misclaffication rate of %f fro sampling ratio of %f and %f", k, missclafication.rate,sampling.rate*100,(1-sampling.rate)*100)
        result2=data.frame(mink=selected.k,min.misclassification.rate=min,train.size=sampling.rate)
        minified.obs <- rbind(minified.obs, result2)
    return(list(observation,minified.obs))
}



In [72]:
n.points <- nrow(credit.data)
# sampling.rate <- 0.7
minified.obs=data.frame(mink=double(),min.misclassification.rate=double(),train.size=double())
observation=data.frame(k=double(),mink=double(),misclassification.rate=double(),train.size=double(),minified.obs)
for(i in seq(0.5, 0.9, by = 0.1)){
#     sprintf("%f rate",i)
     
    x=knnFun(sampling.rate=i,n.points=nrow(credit.data),observation=observation,minified.obs)
    observation=data.frame(x[1])
    minified.obs=data.frame(x[2])
}

In [73]:
observation

k,mink,misclassification.rate,train.size
1,1,43.29502,0.5
2,2,39.08046,0.5
3,2,42.91188,0.5
4,2,41.37931,0.5
5,2,44.44444,0.5
6,2,41.76245,0.5
7,2,43.67816,0.5
8,8,38.31418,0.5
9,8,38.31418,0.5
10,10,35.63218,0.5


In [74]:
minified.obs
minified3=minified.obs

mink,min.misclassification.rate,train.size
16,32.95019,0.5
9,35.40670,0.6
18,36.94268,0.7
17,35.23810,0.8
18,41.50943,0.9


# Observation 3: 


### Applying the min max algorithm for normalization didnot help much. Best we could do was 32.9% with k=16 and sample size of 50%.

In [76]:
print("Minimum value of k that gives minimum missclassification rate for a particular sample size on data with no normalization")
print("-------------------------------------------------------------------------------------------------------")
minified1
print("Minimum value of k that gives minimum missclassification rate for a particular sample size on data with Intutive normalization")
print("-------------------------------------------------------------------------------------------------------")
minified2
print("Minimum value of k that gives minimum missclassification rate for a particular sample size on data with min max normalization")
print("-------------------------------------------------------------------------------------------------------")
minified3

[1] "Minimum value of k that gives minimum missclassification rate for a particular sample size on data with no normalization"
[1] "-------------------------------------------------------------------------------------------------------"


mink,min.misclassification.rate,train.size
8,39.08046,0.5
12,36.36364,0.6
11,35.03185,0.7
5,34.28571,0.8
20,35.84906,0.9


[1] "Minimum value of k that gives minimum missclassification rate for a particular sample size on data with Intutive normalization"
[1] "-------------------------------------------------------------------------------------------------------"


mink,min.misclassification.rate,train.size
11,37.93103,0.5
4,36.84211,0.6
18,31.84713,0.7
18,37.14286,0.8
20,26.41509,0.9


[1] "Minimum value of k that gives minimum missclassification rate for a particular sample size on data with min max normalization"
[1] "-------------------------------------------------------------------------------------------------------"


mink,min.misclassification.rate,train.size
16,32.95019,0.5
9,35.40670,0.6
18,36.94268,0.7
17,35.23810,0.8
18,41.50943,0.9


# Observation : Normalizing the data with intutive normalization technique best classifies the data with misclassification rate as low as 26.43% for sample size = 90% and k=20